## Resumen del problema

Seleccionar un subconjunto de variables que maximice el rendimiento de un algoritmo de ensamble para regresión (por ejemplo, bosques aleatorios), usando un dataset real y evaluando R² (coeficiente de determinación) como medida de aptitud (fitness).

Algoritmos metaheurísticos :

1. Recocido simulado (Simulated Annealing - SA)
2. Algoritmo genético (Genetic Algorithm - GA)
3. Enjambre de partículas (Particle Swarm Optimization, PSO)

### Dataset base para pruebas

Usaremos el dataset diabetes de sklearn.datasets.

### Configuración general antes de los algoritmos

### Importar librerías

In [ ]:
import numpy as np
import pandas as pd

# Utilizada para selección aleatoria de elementos
import random

from sklearn.datasets import load_diabetes
# Carga un conjunto de datos real de diabetes incluido en scikit-learn.
# Este dataset contiene 10 características (variables) que se usarán como base
# para aplicar selección de variables con metaheurísticas.

from sklearn.model_selection import cross_val_score
# Permite evaluar el rendimiento de un modelo con validación cruzada.
# Se usa para medir qué tan buena es una solución (subconjunto de variables),
# calculando la precisión promedio en varias particiones de los datos.

from sklearn.metrics import make_scorer, mean_squared_error
# make_scorer para crear métricas personalizadas compatibles con scikit-learn y
# mean_squared_error para calcular el error cuadrático medio (MSE)

from sklearn.ensemble import RandomForestRegressor
# Algoritmo de regresión RandomForest.
# En este caso, lo usamos como modelo base para evaluar la calidad
# de los subconjuntos de variables seleccionados por los algoritmos.

### Cargar datos reales

In [ ]:
data = load_diabetes()
X = data.data
y = data.target

n_features = X.shape[1]

### Convertir a DataFrame para visualizar el conjunto de datos

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

### Dimensiones del dataset

In [ ]:
print(df.shape)
print('Cantidad de registros:', df.shape[0])
print('Cantidad de variables:', df.shape[1])

### Variables o característas del dataset

In [ ]:
print(data.feature_names)

### Información del dataset

In [ ]:
df.info()

### Función de evaluación (fitness): precisión media con validación cruzada

Esta función es el corazón del problema de optimización: las metaheurísticas (como recocido simulado o algoritmo genético) generan soluciones candidatas (máscaras binarias) y esta función les dice qué tan buena es cada una, basándose en la precisión de clasificación.



In [ ]:
def evaluate_solution(mask):
    # Esta función evalúa qué tan buena es una solución (máscara de selección de variables)
    # La entrada 'mask' es un vector binario del mismo tamaño que el número de características (features).
    # Por ejemplo, si mask = [1, 0, 1, 0], significa que solo se están usando las variables 0 y 2.

    if np.sum(mask) == 0:
        # Si no se selecciona ninguna variable (todos los valores en mask son 0),
        # no se puede entrenar un modelo. En ese caso, se devuelve precisión = 0.
        return 0

    # Selecciona solo las columnas de X donde mask == 1 (es decir, las variables activadas)
    X_selected = X[:, mask == 1]

    # Se elige un modelo RandomForest para regresión con 25 estimadores
    regr = RandomForestRegressor(n_estimators=2, random_state=42)

    # Se evalúa la precisión (R2) promedio usando validación cruzada de 5 pliegues (5-fold cross-validation)
    # Esto ayuda a estimar qué tan bien generaliza la selección de variables sin sobreajuste
    score = cross_val_score(regr, X_selected, y, cv=5).mean()

    # Si se desea cambiar la métrica a RMSE, se tendrá que hacer explícitamente:
    # score = cross_val_score(regr, X_selected, y, scoring='neg_root_mean_squared_error', cv=5).mean()
    # scoring='neg_root_mean_squared_error' devuelve RMSE negativo a propósito, porque sigue la
    # convención de que métricas de error deben ser maximizadas si se invierte el signo.

    # Se retorna la precisión media como la medida de calidad (fitness) de la solución
    return score

### Recocido Simulado (Simulated Annealing, SA)

Este algoritmo imita el proceso físico de enfriamiento de metales, permitiendo al sistema "explorar" soluciones peores al principio (para evitar quedarse atrapado en mínimos locales), y luego refinar progresivamente la búsqueda hasta encontrar una solución óptima o cercana.



In [ ]:
def simulated_annealing(n_iterations=100, initial_temp=1.0, cooling_rate=0.995):
    """
    Implementa el algoritmo de Recocido Simulado (Simulated Annealing) para seleccionar
    un subconjunto de variables que maximice la precisión de un estimador.

    Parámetros:
    - n_iterations: número total de iteraciones a ejecutar.
    - initial_temp: temperatura inicial del sistema (controla la probabilidad de aceptar soluciones peores al inicio).
    - cooling_rate: factor por el cual se reduce la temperatura en cada iteración (debe estar entre 0 y 1).

    Retorna:
    - best: la mejor máscara binaria encontrada (variables seleccionadas).
    - best_score: precisión promedio (fitness) de esa mejor solución.
    """

    # Generar solución inicial aleatoria: vector binario (0 o 1) del mismo tamaño que el número de características
    current = np.random.randint(0, 2, size=n_features)

    # Evaluar la calidad (precisión) de la solución inicial
    current_score = evaluate_solution(current)

    # Guardar la mejor solución conocida hasta el momento (inicialmente es la misma)
    best = current.copy()
    best_score = current_score

    # Establecer temperatura inicial
    temp = initial_temp

    # Ciclo principal del algoritmo (iteraciones)
    for i in range(n_iterations):

        # Crear una solución vecina modificando una sola variable aleatoriamente
        neighbor = current.copy()
        idx = np.random.randint(n_features)  # escoger una posición aleatoria
        neighbor[idx] = 1 - neighbor[idx]    # cambiar de 0→1 o de 1→0

        # Evaluar la nueva solución vecina
        neighbor_score = evaluate_solution(neighbor)

        # Calcular la diferencia en rendimiento (delta)
        delta = neighbor_score - current_score

        # Criterio de aceptación tipo Metropolis:
        # - Si la nueva solución es mejor (delta > 0), se acepta siempre.
        # - Si es peor, se acepta con cierta probabilidad que depende de delta y la temperatura.
        if delta > 0 or np.exp(delta / temp) > np.random.rand():
            current = neighbor
            current_score = neighbor_score

            # Si además esta nueva solución es la mejor de todas, la guardamos
            if current_score > best_score:
                best = current.copy()
                best_score = current_score

        # Reducir la temperatura gradualmente (enfriamiento simulado)
        temp *= cooling_rate

        # Mostrar información de progreso cada 20 iteraciones y en la última
        if i % 20 == 0 or i == n_iterations - 1:
            print(f"Iteración {i}: Precisión = {current_score:.4f}, Temp = {temp:.4f}")
            # print(f"Iteración {i}: Precisión = {-current_score:.4f}, Temp = {temp:.4f}")

    # Al finalizar, se retorna la mejor solución encontrada
    return best, best_score

### Resultados

In [ ]:
best_sa, score_sa = simulated_annealing()

print(f"\nMejor precisión SA: {score_sa:.4f}")
# print(f"\nMejor precisión SA: {-score_sa:.4f}")
print(f"Variables seleccionadas: {np.sum(best_sa)}")

# Mostrar nombres de las variables seleccionadas
feature_names_array = np.array(data.feature_names)
selected_features_sa = feature_names_array[best_sa == 1]
for feature in selected_features_sa:
    print(f"- {feature}")

### Algoritmo Genético (Genetic Algorithm, GA)

¿Qué hace este algoritmo?
* Crea una población inicial de soluciones (máscaras binarias de selección de variables).
* Evalúa su desempeño mediante validación cruzada.
* Selecciona los mejores individuos para formar la siguiente generación.
* Genera nuevos individuos aplicando cruce y mutación.
* Itera por varias generaciones, refinando las soluciones.
* Devuelve el mejor conjunto de variables al final del proceso.



In [ ]:
def genetic_algorithm(pop_size=20, generations=10, crossover_rate=0.8, mutation_rate=0.1):
    """
    Algoritmo genético para seleccionar un subconjunto óptimo de variables que maximice
    el rendimiento de un algoritmo de regresión (en este caso, bosques aleatorios).

    Parámetros:
    - pop_size: número de individuos en la población.
    - generations: número total de generaciones que se van a evolucionar.
    - crossover_rate: probabilidad de realizar cruce entre padres.
    - mutation_rate: probabilidad de mutar un hijo.

    Retorna:
    - best_ind: el mejor individuo (máscara de variables seleccionadas).
    - score: precisión (fitness) de ese individuo.
    """

    # Inicializar población aleatoria
    population = [np.random.randint(0, 2, size=n_features) for _ in range(pop_size)]

    # Seguimiento del mejor encontrado en TODO el proceso
    best_overall = None
    best_overall_score = -np.inf

    for gen in range(generations):
        # 1) Evaluar todos los individuos UNA vez (y almacenar los scores)
        scores = np.array([evaluate_solution(ind) for ind in population])

        # 2) Actualizar mejor de la generación y mejor global
        best_idx_gen = np.argmax(scores)
        best_score_gen = scores[best_idx_gen]

        if best_score_gen > best_overall_score:
            best_overall_score = best_score_gen
            best_overall = population[best_idx_gen].copy()

        # Mostrar progreso usando el score ya calculado (sin reevaluar)
        print(f"Generación {gen + 1}: Mejor precisión = {best_score_gen:.4f}")
        # print(f"Generación {gen + 1}: Mejor precisión = {-best_score_gen:.4f}")

        # 3) Selección: tomar los top-k individuos (puedes ajustar k)
        k = max(2, pop_size // 2)  # por ejemplo, quedarnos con la mitad superior
        top_idx = np.argsort(scores)[-k:]  # índices de los mejores k individuos
        selected = [population[i] for i in top_idx]

        # 4) Reproducción (cruce + mutación) para crear la nueva población
        new_population = []
        while len(new_population) < pop_size:
            # Seleccionar dos padres aleatoriamente entre los 'selected'
            parents = random.sample(selected, 2)

            # Cruce por punto
            if np.random.rand() < crossover_rate:
                point = np.random.randint(1, n_features - 1)
                child1 = np.concatenate((parents[0][:point], parents[1][point:]))
                child2 = np.concatenate((parents[1][:point], parents[0][point:]))
            else:
                # Copias si no hay cruce (hacer copia explícita)
                child1 = parents[0].copy()
                child2 = parents[1].copy()

            # Mutación: aquí aplicamos mutación por gen con probabilidad mutation_rate
            for child in (child1, child2):
                # recorrer genes y mutar con probabilidad mutation_rate
                for g in range(n_features):
                    if np.random.rand() < mutation_rate:
                        child[g] = 1 - child[g]
                new_population.append(child)
                if len(new_population) >= pop_size:
                    break

        # Reemplazar población
        population = new_population[:pop_size]

    # Al final devolver el mejor global y su score (ya guardado)
    return best_overall, best_overall_score

### Resultados

In [ ]:
best_ga, score_ga = genetic_algorithm()

print(f"\nMejor precisión GA: {score_ga:.4f}")
# print(f"\nMejor precisión GA: {-score_ga:.4f}")
print(f"Variables seleccionadas: {np.sum(best_ga)}")

# Mostrar nombres de las variables seleccionadas
feature_names_array = np.array(data.feature_names)
selected_features_ga = feature_names_array[best_ga == 1]
for feature in selected_features_ga:
    print(f"- {feature}")

### Enjambre de partículas (Particle Swarm Optimization, PSO)

* Cada partícula es un vector binario que indica si una característica está incluida (1) o excluida (0).
* El fitness (aptitud) es la precisión promedio de un RandomForestRegressor usando solo esas variables.
* PSO ajusta las posiciones (máscaras) y velocidades de las partículas para maximizar el fitness.


In [ ]:
def binary_pso(n_particles=30, max_iter=10, w=0.7, c1=1.5, c2=1.5):
    """
    Implementación de PSO binario para selección de variables.

    Parámetros:
    - n_particles: número de partículas
    - max_iter: iteraciones máximas
    - w: inercia
    - c1: factor cognitivo (atracción hacia mejor posición personal)
    - c2: factor social (atracción hacia mejor posición global)
    """
    # Inicializar posiciones (0 o 1) y velocidades aleatorias
    positions = np.random.randint(0, 2, size=(n_particles, n_features))
    velocities = np.random.rand(n_particles, n_features)

    # Evaluar fitness inicial
    fitness = np.array([evaluate_solution(p) for p in positions])

    # Mejor posición personal y global
    personal_best_positions = positions.copy()
    personal_best_scores = fitness.copy()
    global_best_idx = np.argmax(fitness)
    global_best_position = positions[global_best_idx].copy()
    global_best_score = fitness[global_best_idx]

    # Iteraciones principales
    for it in range(max_iter):
        iteration_scores = []
        for i in range(n_particles):
            # Actualizar velocidad
            r1 = np.random.rand(n_features)
            r2 = np.random.rand(n_features)
            velocities[i] = (
                w * velocities[i] +
                c1 * r1 * (personal_best_positions[i] - positions[i]) +
                c2 * r2 * (global_best_position - positions[i])
            )

            # Actualizar posición usando función sigmoide (versión binaria)
            sigmoid = 1 / (1 + np.exp(-velocities[i]))
            positions[i] = (np.random.rand(n_features) < sigmoid).astype(int)

            # Evaluar nueva posición
            score = evaluate_solution(positions[i])

            # Agregar el score a la lista iteration_scores
            iteration_scores.append(score)

            # Actualizar mejor personal
            if score > personal_best_scores[i]:
                personal_best_scores[i] = score
                personal_best_positions[i] = positions[i].copy()

                # Actualizar mejor global
                if score > global_best_score:
                    global_best_score = score
                    global_best_position = positions[i].copy()

        # Mejor precisión encontrada en ESTA iteración
        best_iteration_score = max(iteration_scores)
        print(f"Iteración {it+1}/{max_iter} - Mejor precisión: {best_iteration_score:.4f}")
        # print(f"Iteración {it+1}/{max_iter} - Mejor precisión: {-best_iteration_score:.4f}")

    return global_best_position, global_best_score

### Resultados

In [ ]:
best_mask, best_score = binary_pso()

print(f"\nMejor precisión PSO: {best_score:.4f}")
# print(f"\nMejor precisión PSO: {-best_score:.4f}")
print("Variables seleccionadas:", np.sum(best_mask))

feature_names_array = np.array(data.feature_names)
for feat in feature_names_array[best_mask == 1]:
    print("-", feat)